In [20]:
import pandas as pd

In [21]:
imdb_titles_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.basics.tsv'
imdb_ratings_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.ratings.tsv'
imdb_principals_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.principals.tsv'
imdb_crew_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/title.crew.tsv'
imdb_names_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/imdb/name.basics.tsv'
rotten_tomatoes_movies_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/rotten_tomatoes/rotten_tomatoes_movies.csv'
netflix_titles_path = '/Users/antoniam/Desktop/personal/netflix-ratings/data/netflix/netflix_titles.csv'

## IMDB dataset

In [22]:
imdb_titles_df = pd.read_csv(imdb_titles_path, '\t', low_memory=False)
imdb_titles_df.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"


In [ ]:
imdb_ratings_df = pd.read_csv(imdb_ratings_path,'\t')
imdb_ratings_df.head(2)

In [ ]:
imdb_principals_df = pd.read_csv(imdb_principals_path,'\t')
imdb_principals_df.head(2)

In [ ]:
imdb_names_df = pd.read_csv(imdb_names_path,'\t')
imdb_names_df.head(2)

## Rotten Tomatoes Dataset

In [ ]:
rt_df = pd.read_csv(rotten_tomatoes_movies_path)
rt_df.head(2)

## Netflix Dataset

In [ ]:
netflix_df = pd.read_csv(netflix_titles_path)
netflix_df.head(2)

## Tables Create Statements

In [2]:
import psycopg2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [70]:
def create_database():

#     # connect to default database
#     conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=antoniam")
#     conn.set_session(autocommit=True)
#     cur = conn.cursor()
    
#     # create sparkify database with UTF8 encoding
#     cur.execute("""
#     SELECT pg_terminate_backend(pg_stat_activity.pid)
#     FROM pg_stat_activity
#     WHERE pg_stat_activity.datname = 'movies_db';""")
#     cur.execute("DROP DATABASE IF EXISTS movies_db")
#     cur.execute("CREATE DATABASE movies_db WITH ENCODING 'utf8' TEMPLATE template0")

#     # close connection to default database
#     conn.close()    
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=movies_db")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    return cur, conn

In [72]:
conn.close()

In [73]:
cur, conn = create_database()

In [ ]:
create_staging_imdb_titles ="""CREATE TABLE IF NOT EXISTS staging_imdb_titles(
tconst VARCHAR NOT NULL PRIMARY KEY,
titleType VARCHAR,
primaryTitle VARCHAR,
originalTitle VARCHAR,
isAdult VARCHAR,
startYear VARCHAR,
endYear VARCHAR,
runtimeMinutes VARCHAR,
genres VARCHAR);
"""

create_staging_imdb_ratings ="""CREATE TABLE IF NOT EXISTS staging_imdb_ratings(
tconst VARCHAR NOT NULL,
averageRating FLOAT,
numVotes INT);
"""

create_staging_imdb_principals = """CREATE TABLE IF NOT EXISTS staging_imdb_principals(
tconst VARCHAR NOT NULL,
ordering INTEGER NOT NULL,
nconst VARCHAR NOT NULL, 
category VARCHAR NOT NULL,
job VARCHAR NOT NULL,
characters VARCHAR NOT NULL);
"""

create_staging_imdb_crew = """CREATE TABLE IF NOT EXISTS staging_imdb_crew(
tconst VARCHAR NOT NULL,
directors VARCHAR, 
writers VARCHAR);
"""

create_staging_imdb_names = """CREATE TABLE IF NOT EXISTS staging_imdb_names(
nconst VARCHAR NOT NULL PRIMARY KEY,
primaryName VARCHAR NOT NULL,
birthYear VARCHAR,
deathYear VARCHAR,
primaryProfession VARCHAR,
knownForTitles VARCHAR);
"""

create_staging_rotten_tomatoes_titles = """CREATE TABLE IF NOT EXISTS staging_rotten_tomatoes_titles(
rotten_tomatoes_link VARCHAR, 
movie_title VARCHAR,
movie_info VARCHAR,
critics_consensus VARCHAR, 
content_rating VARCHAR, 
genres VARCHAR, 
directors VARCHAR, 
authors VARCHAR,
actors VARCHAR, 
original_release_date DATE, 
streaming_release_date DATE, 
runtime FLOAT,
production_company VARCHAR, 
tomatometer_status VARCHAR, 
tomatometer_rating FLOAT,
tomatometer_count INT, 
audience_status VARCHAR, 
audience_rating FLOAT,
audience_count INT, 
tomatometer_top_critics_count INT,
tomatometer_fresh_critics_count INT, 
tomatometer_rotten_critics_count INT);
"""
create_staging_netflix_titles = """CREATE TABLE IF NOT EXISTS staging_netflix_titles(
show_id VARCHAR NOT NULL PRIMARY KEY,
"type" VARCHAR,
title VARCHAR,
director VARCHAR,
"cast" VARCHAR,
country VARCHAR,
date_added DATE,
release_year INT4,
rating VARCHAR,
duration VARCHAR,
listed_in VARCHAR,
description VARCHAR);
"""

In [27]:
create_temp_table = \
"""CREATE TABLE IF NOT EXISTS temp_netflix_imdb(
    imdb_title_id VARCHAR,
    netflix_title_id VARCHAR);
"""

In [11]:
create_titles = \
"""CREATE TABLE IF NOT EXISTS titles(
  id SERIAL PRIMARY KEY,
  name VARCHAR,
  description VARCHAR,
  year INT4,
  runtime_minutes INT,
  country VARCHAR,
  isAdult BOOLEAN,
  type VARCHAR,
  imdb_avg_score FLOAT,
  imdb_n_ratings INT,
  rt_critics_score FLOAT,
  rt_n_critics INT,
  rt_audience_score FLOAT,
  rt_n_audience INT);
"""

create_roles = \
"""CREATE TABLE IF NOT EXISTS roles(
id SERIAL PRIMARY KEY,
title_id VARCHAR,
person_id VARCHAR,
role_name VARCHAR NOT NULL
);
"""

create_persons = \
"""CREATE TABLE IF NOT EXISTS persons(
id VARCHAR NOT NULL PRIMARY KEY,
full_name VARCHAR NOT NULL,
birth_year INT4,
death_year INT4
);
"""

create_genres = \
"""CREATE TABLE IF NOT EXISTS genres(
id SERIAL PRIMARY KEY,
title_id VARCHAR NOT NULL,
genre_name VARCHAR NOT NULL);"""


In [ ]:
staging_tables_sql = [create_staging_imdb_titles, create_staging_imdb_ratings, create_staging_imdb_principals, create_staging_imdb_names, 
 create_staging_rotten_tomatoes_titles, create_staging_netflix_titles, create_staging_imdb_crew, create_temp_table]

final_tables = [create_titles, create_roles, create_persons, create_genres]

In [ ]:
for s in staging_tables_sql + final_tables:
    cur.execute(s)

In [ ]:
copy_staging_titles = f"COPY staging_imdb_titles FROM '{imdb_titles_path}' delimiter '\t' CSV HEADER"
copy_staging_ratings = f"COPY staging_imdb_ratings FROM '{imdb_ratings_path}' delimiter '\t' CSV HEADER"
copy_staging_principals = f"COPY staging_imdb_principals FROM '{imdb_principals_path}' delimiter '\t' CSV HEADER"
copy_staging_crew = f"COPY staging_imdb_crew FROM '{imdb_crew_path}' delimiter '\t' CSV HEADER"
copy_staging_names = f"COPY staging_imdb_names FROM '{imdb_names_path}' delimiter '\t' CSV HEADER"
copy_staging_rt = f"COPY staging_rotten_tomatoes_titles FROM '{rotten_tomatoes_movies_path}' delimiter ',' CSV HEADER"
copy_staging_netflix = f"COPY staging_netflix_titles FROM '{netflix_titles_path}' delimiter ',' CSV HEADER"


copy_staging = [copy_staging_titles, copy_staging_ratings, copy_staging_principals, copy_staging_names, copy_staging_crew, copy_staging_rt, copy_staging_netflix]

for s in copy_staging: 
    print(s)
    cur.execute(s)

## Final tables

### Temp table

In [28]:
cur.execute('DROP table temp_netflix_imdb')

In [29]:
cur.execute(create_temp_table)

In [30]:
from time import time

In [31]:
t = time()
st = \
"""INSERT INTO temp_netflix_imdb(imdb_title_id, netflix_title_id)
SELECT distinct imdb.tconst, netflix.show_id
FROM staging_netflix_titles netflix 
LEFT JOIN staging_imdb_titles imdb
ON lower(substr(netflix.title, 1, 8)) = lower(substr(imdb.primaryTitle,1, 8))
"""
cur.execute(st)
print(time() - t)

76.99673104286194


In [33]:
cur.execute('SELECT * from temp_netflix_imdb limit 5')
cur.fetchone()

('tt0149582', 's295')

### Genres table

In [35]:
cur.execute(create_genres)

In [36]:
st = \
"""INSERT INTO genres(title_id, genre_name)
   SELECT tconst, unnest(string_to_array(genres, ',')) as genre
   FROM temp_netflix_imdb
   LEFT JOIN staging_imdb_titles
   ON temp_netflix_imdb.imdb_title_id = staging_imdb_titles.tconst
   WHERE genres!='\\N'
   """
cur.execute(st)

### Persons table

In [37]:
st = \
"""INSERT INTO persons(id, full_name, birth_year, death_year)
SELECT nconst, primaryName, 
NULLIF(regexp_replace(birthYear, '\D','','g'), '')::numeric AS birth_year,
NULLIF(regexp_replace(deathYear, '\D','','g'), '')::numeric AS death_year
from staging_imdb_names
"""
cur.execute(st)

### Roles table

In [38]:
st = \
"""INSERT INTO roles(title_id, person_id, role_name)
SELECT tconst, nconst, category
FROM temp_netflix_imdb 
LEFT JOIN staging_imdb_principals 
ON temp_netflix_imdb.imdb_title_id = staging_imdb_principals.tconst
WHERE category in ('director', 'writer', 'actor', 'actress')

UNION
SELECT tconst, unnest(string_to_array(directors, ',')) as nconst, 'director'
FROM temp_netflix_imdb 
LEFT JOIN staging_imdb_crew
ON temp_netflix_imdb.imdb_title_id = staging_imdb_crew.tconst
WHERE directors!='\\N'

UNION
SELECT tconst, unnest(string_to_array(writers, ',')) as nconst, 'writer'
FROM temp_netflix_imdb 
LEFT JOIN staging_imdb_crew
ON temp_netflix_imdb.imdb_title_id = staging_imdb_crew.tconst
WHERE writers!='\\N'
"""
cur.execute(st)

### Titles table

In [ ]:
name VARCHAR,
description VARCHAR,
year INT4,
runtime_minutes INT,
country VARCHAR,
isAdult BOOLEAN,
type VARCHAR,
imdb_avg_score FLOAT,
imdb_n_ratings INT,
rt_critics_score FLOAT,
rt_n_critics INT,
rt_audience_score FLOAT,
rt_n_audience INT

CREATE TABLE IF NOT EXISTS staging_imdb_titles(
tconst VARCHAR NOT NULL PRIMARY KEY,
titleType VARCHAR,
primaryTitle VARCHAR,
originalTitle VARCHAR,
rating VARCHAR,
startYear VARCHAR,
endYear VARCHAR,
runtimeMinutes VARCHAR,
genres VARCHAR);

In [152]:
t = time()
st = """
SELECT staging_netflix_titles.show_id, staging_netflix_titles.type, staging_netflix_titles.title, 
staging_netflix_titles.country, staging_netflix_titles.release_year, 
staging_netflix_titles.description, with_imdb.tconst, staging_netflix_titles.rating, with_imdb.runtimeMinutes, 
averageRating, numVotes

FROM staging_netflix_titles 
LEFT JOIN (
        SELECT distinct netflix.show_id, netflix.type, netflix.title, netflix.country, netflix.release_year, 
                netflix.description, imdb.tconst, runtimeMinutes, averageRating, numVotes
                FROM(
                    SELECT show_id, type, title, country, release_year, description,
                    unnest(string_to_array(staging_netflix_titles.director, ',')) as director,
                    unnest(string_to_array(staging_netflix_titles.cast, ',')) as actor
                    FROM staging_netflix_titles) as netflix
                JOIN(
                    SELECT *
                    FROM temp_netflix_imdb 
                    JOIN staging_imdb_titles 
                    ON temp_netflix_imdb.imdb_title_id = staging_imdb_titles.tconst
                    JOIN roles
                    ON roles.title_id = staging_imdb_titles.tconst
                    JOIN persons 
                    ON persons.id = roles.person_id) as imdb
                JOIN staging_imdb_ratings 
                    ON staging_imdb_ratings.tconst = imdb.tconst
            ON netflix.show_id = imdb.netflix_title_id
            AND netflix.release_year::varchar = imdb.startYear
            AND (netflix.actor = imdb.full_name or netflix.director = imdb.full_name)
        ) as with_imdb
ON with_imdb.show_id = staging_netflix_titles.show_id
"""
cur.execute(st)

In [179]:
t = 0
for i in range(500):
    if cur.fetchone()[-1] is not None:
        t+=1

In [150]:
cur.fetchone()

('s2',
 'Movie',
 '7:19',
 'Mexico',
 2016,
 'After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive.',
 'tt5979312',
 'TV-MA',
 '94')

In [147]:
cur.fetchone()

(None, None, None, None, None, None, None, None, None)

In [97]:
t = time()
st = """
SELECT distinct netflix.show_id, rt.rotten_tomatoes_link, imdb.tconst
FROM temp_netflix_imdb
LEFT JOIN staging_netflix_titles netflix
ON temp_netflix_imdb.netflix_title_id = netflix.show_id

LEFT JOIN (
SELECT * FROM staging_imdb_titles imdb
JOIN temp_netflix_imdb
ON temp_netflix_imdb.imdb_title_id = imdb.tconst
JOIN (
    SELECT full_name, title_id
    FROM roles
    JOIN PERSONS 
    ON roles.person_id = persons.id) as imdb_cast
ON imdb_cast.
FULL OUTER JOIN (
    SELECT staging_netflix_titles.show_id, unnest(string_to_array(staging_netflix_titles.director, ',')) as name
    FROM staging_netflix_titles) as director
ON director.show_id = netflix.show_id
AND director.name = imdb_cast.full_name
AND temp_netflix_imdb.imdb_title_id = imdb_cast.title_id
AND netflix.release_year::varchar = imdb.startYear



LEFT JOIN staging_rotten_tomatoes_titles rt
ON lower(substr(netflix.title, 1, 8)) = lower(substr(rt.movie_title, 1, 8))
AND netflix.release_year::varchar = extract(year from rt.original_release_date)::varchar
"""
res = cur.execute(st)
print(time() - t)

32.14161825180054


In [98]:
cur.fetchall()

[('s1', None, None),
 ('s10', None, 'tt1301698'),
 ('s10', None, None),
 ('s100', None, None),
 ('s1000', None, 'tt3748440'),
 ('s1000', None, 'tt6615138'),
 ('s1000', None, None),
 ('s1001', None, 'tt6488384'),
 ('s1001', None, 'tt6769208'),
 ('s1001', None, 'tt6957370'),
 ('s1001', None, 'tt7551534'),
 ('s1001', None, None),
 ('s1002', None, 'tt2359024'),
 ('s1002', None, None),
 ('s1003', 'm/blue_streak', 'tt0181316'),
 ('s1003', 'm/blue_streak', None),
 ('s1004', 'm/blue_valentine', 'tt1120985'),
 ('s1004', 'm/blue_valentine', 'tt4677834'),
 ('s1004', 'm/blue_valentine', None),
 ('s1005', None, 'tt0476527'),
 ('s1006', None, 'tt10754264'),
 ('s1006', None, 'tt4510472'),
 ('s1006', None, 'tt5602204'),
 ('s1006', None, 'tt5858502'),
 ('s1006', None, 'tt6533314'),
 ('s1006', None, 'tt7282932'),
 ('s1006', None, 'tt7407132'),
 ('s1006', None, 'tt7444956'),
 ('s1006', None, 'tt7912956'),
 ('s1006', None, None),
 ('s1007', None, 'tt12013558'),
 ('s1008', None, 'tt8730152'),
 ('s1009', No

In [60]:
res = cur.fetchall()

In [65]:
len(set(res))

6847

In [ ]:
imdb_titles_df[imdb_titles_df['tconst']=='tt5979312']

In [ ]:
[t for t in list(netflix_df.show_id) if t not in v]

In [ ]:
imdb_titles_df[imdb_titles_df.tconst == 'tt0038171']

In [ ]:
netflix_df[netflix_df.show_id=='s7073']

In [ ]:
(SELECT id, unnest(string_to_array(news_article.news_category_id, ',')):: int rowz
      FROM news_article) AS c ON c.id = a.id

In [ ]:
cur.execute("SELECT staging_netflix_titles.show_id, unnest(string_to_array(staging_netflix_titles.director, ',')) from staging_netflix_titles")

In [ ]:
st = """SELECT COUNT(*) 
FROM temp_netflix_imdb
JOIN staging_netflix_titles netflix
ON temp_netflix_imdb.netflix_title_id = netflix.show_id
JOIN staging_imdb_titles imdb
ON temp_netflix_imdb.imdb_title_id = imdb.tconst
WHERE netflix.release_year::varchar = imdb.startYear"""
cur.execute(st)

In [ ]:
cur.fetchone()

In [ ]:
cur.execute("SELECT levenshtein('GUMBO', 'GAMBOL')")
cur.fetchone()